Wat betekent “vector transformation”?

Tot nu toe hebben we tekst omgezet naar vectoren (BoW, TF-IDF).
Maar vaak moeten we deze vectoren transformeren, zodat ze:

- geschikt worden voor machine learning modellen,
- gestandaardiseerd of verkleind worden om beter te presteren.

Wat kunnen we doen met vectoren?
 Normalisatie / Schalen

We zorgen dat vectoren een vergelijkbare lengte hebben.
Bij TF-IDF is dit vaak al genormaliseerd, maar bij BoW niet.

In [1]:
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import CountVectorizer

texts = ["I love NLP", "I hate NLP"]

vectorizer = CountVectorizer()
bow_vectors = vectorizer.fit_transform(texts).toarray()

from sklearn.preprocessing import Normalizer
norm = Normalizer()
normalized = norm.fit_transform(bow_vectors)

print("Before normalization:\n", bow_vectors)
print("\nAfter normalization:\n", normalized)


Before normalization:
 [[0 1 1]
 [1 0 1]]

After normalization:
 [[0.         0.70710678 0.70710678]
 [0.70710678 0.         0.70710678]]


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer

# ===============================
# 1️⃣ De originele teksten
# ===============================
texts = ["I love NLP", "I hate NLP"]

# ===============================
# 2️⃣ Maak de vectorizer aan
# ===============================
vectorizer = CountVectorizer()

# ===============================
# 3️⃣ Pas de vectorizer toe op de teksten
#     (fit = leer het vocabulaire)
#     (transform = maak de numerieke vectoren)
# ===============================
bow_vectors = vectorizer.fit_transform(texts).toarray()

# ===============================
# 4️⃣ Bekijk het vocabulaire
# ===============================
print("Vocabulary (woordenlijst):", vectorizer.get_feature_names_out())
# Output: ['hate' 'i' 'love' 'nlp']

# ===============================
# 5️⃣ Bekijk de Bag-of-Words matrix
# ===============================
print("\nBag-of-Words matrix (voor normalisatie):")
print(bow_vectors)

# De matrix ziet er zo uit:
# [[0 1 1 1]   ← "I love NLP"
#  [1 1 0 1]]  ← "I hate NLP"

# Betekenis per kolom:
#   hate | i | love | nlp
#   0      1    1      1   → "I love NLP"
#   1      1    0      1   → "I hate NLP"
#
# 1 betekent: woord komt voor in de zin
# 0 betekent: woord komt NIET voor

# ===============================
# 6️⃣ Normaliseren van vectoren
# ===============================
norm = Normalizer()
normalized = norm.fit_transform(bow_vectors)

print("\nNa normalisatie:")
print(normalized)

# Normalisatie zorgt dat elke rij (documentvector)
# dezelfde lengte krijgt.
# Het verandert de schaal (de grootte van de getallen)
# maar niet de verhoudingen tussen woorden.


Vocabulary (woordenlijst): ['hate' 'love' 'nlp']

Bag-of-Words matrix (voor normalisatie):
[[0 1 1]
 [1 0 1]]

Na normalisatie:
[[0.         0.70710678 0.70710678]
 [0.70710678 0.         0.70710678]]


Dimensionality Reduction (optioneel)

Soms heb je duizenden woorden, dus ook duizenden features.
Dat is zwaar voor het model → we kunnen de dimensie verkleinen.

Bijv. met TruncatedSVD (Latent Semantic Analysis):

In [3]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

docs = [
    "Machine learning is fun",
    "Deep learning drives AI",
    "I love studying AI and machine learning"
]

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(docs)

svd = TruncatedSVD(n_components=2)
X_reduced = svd.fit_transform(X)

print("Original shape:", X.shape)
print("Reduced shape:", X_reduced.shape)


Original shape: (3, 10)
Reduced shape: (3, 2)


Wat betekent “Dimensionality Reduction”?

Wanneer je tekst omzet in cijfers (zoals met Bag-of-Words of TF-IDF), krijg je vaak duizenden kolommen —
elke kolom is een woord in je dataset.


| Tekst                        | love | hate | NLP | AI | deep | learning | model | accuracy |
| ---------------------------- | ---- | ---- | --- | -- | ---- | -------- | ----- | -------- |
| "I love NLP"                 | 1    | 0    | 1   | 0  | 0    | 0        | 0     | 0        |
| "I hate NLP"                 | 0    | 1    | 1   | 0  | 0    | 0        | 0     | 0        |
| "AI model improves accuracy" | 0    | 0    | 0   | 1  | 0    | 0        | 1     | 1        |





Hier heb je 8 kolommen (dimensies) —
maar in een echte dataset kunnen dat 10.000 woorden zijn → 10.000 dimensies!

Dat is:
zwaar voor je computer
moeilijk voor modellen om mee te werken
en veel woorden lijken op elkaar (bijv. love, like, enjoy)

Dus we willen die informatie samenvatten → met minder dimensies, maar zonder te veel betekenis te verliezen.
Dat proces heet dimensionality reduction.



Je kunt het vergelijken met samenvatten van een lange tekst:
In plaats van 10.000 woorden te onthouden, wil je 2 of 3 belangrijke thema’s vinden.
Bijvoorbeeld: “positieve emotie”, “negatieve emotie”, “AI-technisch”.
Dat doen we wiskundig met methodes zoals:
TruncatedSVD (Latent Semantic Analysis – LSA)
PCA (Principal Component Analysis)
UMAP / t-SNE (voor visualisaties)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd

# 1️⃣ Stel we hebben meerdere documenten
texts = [
    "I love NLP and machine learning",
    "I hate spam emails",
    "Deep learning improves NLP",
    "AI models learn from data",
    "Spam detection with machine learning"
]

# 2️⃣ Zet ze om naar TF-IDF vectoren
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

print("Aantal woorden (dimensies):", X.shape[1])

# 3️⃣ Verminder de dimensies met TruncatedSVD
svd = TruncatedSVD(n_components=2)  # we houden maar 2 'thema’s' over
X_reduced = svd.fit_transform(X)

# 4️⃣ Bekijk het resultaat
df = pd.DataFrame(X_reduced, columns=["Thema 1", "Thema 2"])
print("\nGereduceerde representatie:")
print(df)


Aantal woorden (dimensies): 17

Gereduceerde representatie:
        Thema 1       Thema 2
0  7.654210e-01 -2.767374e-01
1  2.667281e-01  8.337300e-01
2  6.355989e-01 -4.234057e-01
3 -4.281756e-16 -1.788692e-15
4  6.849050e-01  3.775087e-01
